In [1]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, TimeDistributed, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split





In [2]:
# Function to load data and duplicate original videos
def load_data(video_dir, labels, num_frames=10, img_size=(128, 128), max_fake_videos=100):
    videos = []
    video_labels = []
    for label in labels:
        folder = os.path.join(video_dir, label)
        fake_videos_counter = 0  # Counter for fake videos
        for video_name in os.listdir(folder):
            video_path = os.path.join(folder, video_name)
            frames = []
            cap = cv2.VideoCapture(video_path)
            frame_count = 0
            while cap.isOpened() and frame_count < num_frames:
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, img_size)  # Resize frame
                frames.append(frame)
                frame_count += 1
            cap.release()
            if len(frames) == num_frames:  # Ensure the video has enough frames
                if label == 'DFD_manipulated_sequences' and fake_videos_counter >= max_fake_videos:
                    continue  # Skip additional fake videos if we have 300 already
                videos.append(np.array(frames))
                video_labels.append(0 if label == 'DFD_original_sequences' else 1)  # 0 for real, 1 for fake
                if label == 'DFD_manipulated_sequences':
                    fake_videos_counter += 1
                        # Duplicate original videos (real class)
    videos = np.array(videos)
    video_labels = np.array(video_labels)
    videos_real = videos[video_labels == 0]
    video_labels_real = video_labels[video_labels == 0]

    # Duplicate the real class videos to balance the dataset
    videos = np.concatenate([videos, videos_real])
    video_labels = np.concatenate([video_labels, video_labels_real])

    return videos, video_labels

In [ ]:
# Load data
# video_dir = r"your folder containing original and manipulated "
labels = ['DFD_original sequences', 'DFD_manipulated_sequences']

# Assuming load_data is defined elsewhere to load the video data and labels
# Ensure the function returns the data and labels correctly formatted
X, y = load_data(video_dir, labels, num_frames=10, img_size=(128, 128))

# Normalize pixel values to the range [0, 1]
X = X / 255.0  

# Convert labels to one-hot encoding with 2 classes
y = tf.keras.utils.to_categorical(y, num_classes=2)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, LSTM, Dense, TimeDistributed, Dropout, BatchNormalization
)

In [5]:
# Define the model
def build_model(input_shape):
    model = Sequential()

    # TimeDistributed CNN Layers for spatial feature extraction
    model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=input_shape))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(BatchNormalization()))

    model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(BatchNormalization()))

    model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Flatten()))

    # LSTM Layers for temporal sequence learning
    model.add(LSTM(128, return_sequences=False, dropout=0.2, recurrent_dropout=0.2))

    # Fully connected layers
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(2, activation='softmax'))  # 2 classes: real or fake

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
input_shape = (10, 128, 128, 3)  # 10 frames, 128x128 images, 3 color channels

# Build the model
model = build_model(input_shape)
model.summary()

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=16,
    verbose=1
)


c:\Users\MOVIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, 10, 126, 126,   │           896 │
│ (TimeDistributed)               │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 10, 63, 63, 32) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 10, 63, 63, 32) │           128 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 10, 61, 61, 64) │        18,496 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 10, 30, 30, 64) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, 10, 30, 30, 64) │           256 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_6              │ (None, 10, 28, 28,     │        73,856 │
│ (TimeDistributed)               │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_7              │ (None, 10, 14, 14,     │             0 │
│ (TimeDistributed)               │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_8              │ (None, 10, 25088)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │    12,911,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,029,634 (49.70 MB)

 Trainable params: 13,029,442 (49.70 MB)

 Non-trainable params: 192 (768.00 B)

Epoch 1/2
19/19 ━━━━━━━━━━━━━━━━━━━━ 68s 3s/step - accuracy: 0.9098 - loss: 0.2686 - val_accuracy: 1.0000 - val_loss: 0.0168
Epoch 2/2
19/19 ━━━━━━━━━━━━━━━━━━━━ 53s 3s/step - accuracy: 0.9951 - loss: 0.0451 - val_accuracy: 1.0000 - val_loss: 0.0053


In [11]:


def preprocess_video(video_path, num_frames=10, img_size=(128, 128)):
    """
    Preprocess the video by extracting frames and resizing them.
    Args:
        video_path (str): Path to the video file.
        num_frames (int): Number of frames to extract.
        img_size (tuple): Desired image size (height, width).
    Returns:
        np.array: Preprocessed frames (num_frames, height, width, channels).
    """
    frames = []
    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while cap.isOpened() and frame_count < num_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, img_size)  # Resize frame
        frames.append(frame)
        frame_count += 1
    cap.release()

    # Ensure the video has enough frames
    if len(frames) == num_frames:
        return np.array(frames) / 255.0  # Normalize the frames to the range [0, 1]
    else:
        return None

# Path to the test video
video_path = r"C:\Users\MOVIN\Downloads\archive (13)\DFD_manipulated_sequences\DFD_manipulated_sequences\28_16__walking_down_street_outside_angry__ZA8XJIE9.mp4"

# Preprocess the video
video_data = preprocess_video(video_path)

if video_data is not None:
    # Reshape for model input: (1, num_frames, height, width, channels)
    video_data = np.expand_dims(video_data, axis=0)

    # Predict using the trained model
    prediction = model.predict(video_data)
    predicted_class = np.argmax(prediction, axis=1)

    # Output the prediction
    if predicted_class == 0:
        print("The video is classified as: Real")
    else:
        print("The video is classified as: Fake")
else:
    print("The video does not contain enough frames.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
The video is classified as: Fake
